In [16]:
using DelimitedFiles

include("energy_loss/energy_loss.jl")
include("proton_flux/proton_flux.jl")
include("particle_propagation/numerical_propagation.jl")

E_g (generic function with 1 method)

# Interações com fótons de fundo

### Produção de pares

In [4]:
xi = range(0.4, 4, 100)
xi = 10 .^(xi)

phi_xi = phi.(xi)

open("outputs/phi_xi.txt", "w") do io
    writedlm(io, ["xi" "phi_xi"])
    writedlm(io, [xi phi_xi])
end

In [5]:
nu = range(-3, 2, 100)
nu = 10 .^ (nu)

fnu = f_nu.(nu)

open("outputs/f_nu.txt", "w") do io
    writedlm(io, ["nu" "f_nu"])
    writedlm(io, [nu fnu])
end

### Todas as interações

In [2]:
E = range(17, 22, 1000)
E = 10 .^ (E)

beta_pair_E = beta_pair.(E)
beta_pion_E = beta_pion.(E)
beta_rsh_E = fill(beta_rsh(), 1000)
beta_total_E = beta.(E, 0)

open("outputs/beta_E.txt", "w") do io
    writedlm(io, ["E" "beta_pair_E" "beta_pion_E" "beta_rsh_E" "beta_total_E"])
    writedlm(io, [E beta_pair_E beta_pion_E beta_rsh_E beta_total_E])
end

In [5]:
E = range(17, 22, 1000)
E = 10 .^ (E)

db0_pair_dE_E = db0_pair_dE.(E)
db0_pion_dE_E = db0_pion_dE.(E)
db0_rsh_dE_E = fill(db0_rsh_dE(), 1000)
db0_dE_E = db0_dE.(E)

open("outputs/dbeta_dE_E.txt", "w") do io
    writedlm(io, ["E" "db0_pair_dE" "db0_pion_dE" "db0_rsh_dE" "db0_dE"])
    writedlm(io, [E db0_pair_dE_E db0_pion_dE_E db0_rsh_dE_E db0_dE_E])
end

# Propagação de partículas

Resolve a equação

$$
\frac{dE}{dt} = -E \frac{dt}{dz}\beta(E, z)
$$

utilizando de condições iniciais $z = z_{src}$ e $E = E_{src}$ até $z = 0$.

In [6]:
include("particle_propagation/numerical_propagation.jl")

E = [1e18 1e19 1e20 1e21]

for (i, E_src) in enumerate(E)
    
    open("outputs/one_particle_propagation/E_t_$i.txt", "w") do io

        x, y = proton_propa(E_src, 0.4)

        writedlm(io, ["z" "E"])
        writedlm(io, [x y])
            
    end
end

A partir da equação acima, utilizando de condições iniciais $z = 0$ e $E = E$, é mapeado a energia de geração $E_g(E, z_g)$ até $z = z_{src}$.

In [7]:
include("particle_propagation/numerical_propagation.jl")

z_srcs = [0.01 0.05 0.5 1]

E = range(17, 21, 100)
E = 10 .^(E)

for (i, z_src) in enumerate(z_srcs)
    
    open("outputs/generation_energy/E_g_$i.txt", "w") do io

        Eg = E_g.(E, z_src)

        writedlm(io, ["E" "Eg"])
        writedlm(io, [E Eg])
            
    end
end

Calcula a integral de mapeamento de um intervalo de energia da fonte com a Terra

$$
\frac{dE_g}{dE} = (1 + z_g)\exp{\left[\frac{1}{H_0}\int^{z_g}_0dz(1 + z)^{1/2}\left(\frac{db_0(E)}{dE}\right)_{E = E_g(z)(1 + z)}\right]}
$$

In [ ]:
include("proton_flux/proton_flux.jl")

z_srcs = [7.7E-3 0.04 8.5e-2]

E = range(17, 21, 50)
E = 10 .^(E)

for (i, z) in enumerate(z_srcs)
    
    open("outputs/energy_connection/dEsrc_dE_$i.txt", "w") do io
        
        dEsrc_dE_z = dEg_dE.(z, E)

        writedlm(io, ["E" "dEsrc_dE"])
        writedlm(io, [E dEsrc_dE_z])
    end
end 

# Fator de modificação

In [8]:
E = range(17, 21, 100)
E = 10 .^ (E)

z_srcs = [3.8e-3 7.7e-3 2.4e-2 4E-2 8.5E-2 0.19 0.33 0.51 0.76 1.17]

for (i, z_mod) in enumerate(z_srcs)
    
    open("outputs/modification_factor/mod_factor_E_$i.txt", "w") do io

        modification_factor = mod_factor.(E, z_mod)
        
        writedlm(io, ["E" "mod_factor"])
        writedlm(io, [E modification_factor])
    end
end

Resolvendo para muitas fontes

In [ ]:
E = range(17, 21, 50)
E = 10 .^(E)

z_max = [3.8e-3 7.7e-3 2.4e-2 4E-2 8.5E-2 0.19 0.33 0.51 0.76 1.17]

for (i, z_g) in enumerate(z_max)
    
    open("outputs/modification_factor_msources/mod_factor_E_$i.txt", "w") do io

        modification_factor = mod_factor_multiple_sources.(E, z_g)
        
        writedlm(io, ["E" "mod_factor"])
        writedlm(io, [E modification_factor])
    end
end

E = 1.0e17
E = 1.2067926406393264e17
E = 1.4563484775012384e17
E = 1.7575106248547965e17
E = 2.1209508879201926e17
E = 2.559547922699533e17
E = 3.088843596477472e17
E = 3.727593720314953e17
E = 4.4984326689694534e17
E = 5.428675439323859e17
E = 6.551285568595496e17
E = 7.90604321090767e17
E = 9.540954763499963e17
E = 1.1513953993264481e18
E = 1.3894954943731361e18
E = 1.6768329368110031e18
E = 2.0235896477251638e18
E = 2.4420530945486546e18
E = 2.9470517025518095e18
E = 3.5564803062231214e18
E = 4.291934260128761e18
E = 5.179474679231223e18
E = 6.250551925273976e18
E = 7.543120063354608e18
E = 9.102981779915189e18
E = 1.0985411419875617e19
E = 1.3257113655901108e19
E = 1.5998587196060574e19
E = 1.9306977288832455e19
E = 2.3299518105153815e19
E = 2.8117686979742368e19
E = 3.3932217718953296e19
E = 4.0949150623804195e19
E = 4.941713361323818e19
E = 5.963623316594661e19
E = 7.196856730011528e19
E = 8.68511373751352e19
E = 1.0481131341546832e20
E = 1.264855216855301e20
E = 1.52641796717523

sys:1: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.


E = 5.963623316594661e19
E = 7.196856730011528e19
E = 8.68511373751352e19
E = 1.0481131341546832e20
E = 1.264855216855301e20
E = 1.5264179671752366e20
E = 1.8420699693267164e20
E = 2.222996482526191e20
E = 2.6826957952797164e20
E = 3.237457542817653e20
E = 3.906939937054621e20
E = 4.71486636345739e20
E = 5.6898660290182814e20
E = 6.866488450043026e20
E = 8.28642772854686e20
E = 1.0e21
E = 1.0e17
E = 1.2067926406393264e17
E = 1.4563484775012384e17
E = 1.7575106248547965e17
E = 2.1209508879201926e17
E = 2.559547922699533e17
E = 3.088843596477472e17
E = 3.727593720314953e17
E = 4.4984326689694534e17
E = 5.428675439323859e17
E = 6.551285568595496e17
E = 7.90604321090767e17
E = 9.540954763499963e17
E = 1.1513953993264481e18
E = 1.3894954943731361e18
E = 1.6768329368110031e18
E = 2.0235896477251638e18
E = 2.4420530945486546e18
E = 2.9470517025518095e18
E = 3.5564803062231214e18
E = 4.291934260128761e18
E = 5.179474679231223e18
E = 6.250551925273976e18
E = 7.543120063354608e18
E = 9.102981779